# Pruebas para verificar el cumplimiento de los objetivos específicos

## Configuración Inicial y Autenticación

In [ ]:
import os
import requests
import time
import pandas as pd # Opcional, para análisis de datos
from dotenv import load_dotenv

load_dotenv()

# configuration
CLIENT_ID = os.environ.get('CLIENT_ID')
CLIENT_SECRET = os.environ.get('CLIENT_SECRET')

# TODO Endpoints de la API de Spotify a probar (ejemplos)
ENDPOINTS_A_PROBAR = [
    # search
    "https://api.spotify.com/v1/search?q=artist:Adele&type=artist",
    "https://api.spotify.com/v1/search?q=remaster%2520track%3ADoxy%2520artist%3AMiles%2520Davis&type=album",
    # get album
    "https://api.spotify.com/v1/albums/4aawyAB9vmqN3uQ7FjRGTy",
    # get artist
    "https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg",
    # get playlist
    "https://api.spotify.com/v1/playlists/3cEYpjA9oz9GiPac4AsH4n",
    # get track
    "https://api.spotify.com/v1/tracks/11dFghVXANMlKmJXsNCbNl",
    # browse new releases
    "https://api.spotify.com/v1/browse/new-releases",
    # TODO Puedes añadir más endpoints aquí
]

# function to get the access token
def get_access_token(client_id, client_secret):
    token_url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    try:
        response = requests.post(token_url, headers=headers, data=data)
        response.raise_for_status() # Lanza una excepción para errores HTTP
        return response.json()["access_token"]
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener el token de acceso: {e}")
        return None

# Obtener el token de acceso al inicio
ACCESS_TOKEN = get_access_token(CLIENT_ID, CLIENT_SECRET)

if not ACCESS_TOKEN:
    print("No se pudo obtener el token de acceso. Las pruebas no pueden continuar.")
    exit()

HEADERS = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "Content-Type": "application/json"
}

# variables to store the results of the tests
results_objective_a = []
results_objective_b = []
results_objective_c = []

# Objetivo A: Calcular el Umbral Máximo de Peticiones (HTTP 429)

Este objetivo busca encontrar el límite antes de recibir errores HTTP 429 (Too Many Requests). Realizaremos peticiones repetitivas a un endpoint específico, aumentando la frecuencia hasta que aparezcan los errores.

Métricas a Medir para el Objetivo A:

- Tasa de éxito en peticiones: $ \frac{\text{Peticiones exitosas}}{\text{Total de peticiones}} $
- Tasa de error en peticiones (429): $ \frac{\text{Peticiones con error 429}}{\text{Total de peticiones}} $
- Tiempo entre peticiones que devolvieron error (4xx): Registra los intervalos de tiempo entre errores para entender patrones de rate limiting.

In [ ]:
def test_rate_limit(endpoint, headers, initial_delay=0.1, max_requests=500):
    print(f"\n--- Probando umbral de peticiones para: {endpoint} ---")
    num_requests = 0
    successful_requests = 0
    error_429_requests = 0
    other_error_requests = 0
    last_error_time = None
    error_intervals = []

    current_delay = initial_delay

    while num_requests < max_requests:
        start_time = time.time()
        try:
            response = requests.get(endpoint, headers=headers)
            request_duration = time.time() - start_time
            num_requests += 1

            status_code = response.status_code

            if 200 <= status_code < 300:
                successful_requests += 1
                # Reducir el delay si todo va bien para aumentar la tasa de peticiones
                current_delay = max(0.01, current_delay * 0.95)
            elif status_code == 429:

                error_429_requests += 1

                print(f"[{num_requests}] ERROR 429 (Too Many Requests) en {endpoint} - Tiempo: {request_duration:.4f}s")
				
                if last_error_time:
                    error_intervals.append(time.time() - last_error_time)
                last_error_time = time.time()
            else:
                other_error_requests += 1
                print(f"[{num_requests}] Otro ERROR ({status_code}) en {endpoint} - Tiempo: {request_duration:.4f}s")
                if last_error_time:
                    error_intervals.append(time.time() - last_error_time)
                last_error_time = time.time()
                # Aumentar un poco el delay para otros errores también
                current_delay *= 1.1
                time.sleep(current_delay)

            # Pequeña pausa para no sobrecargar el sistema local si no es un 429
            if status_code != 429:
                time.sleep(current_delay)

        except requests.exceptions.RequestException as e:
            num_requests += 1
            other_error_requests += 1
            print(f"[{num_requests}] Excepción de red: {e} en {endpoint}")
            if last_error_time:
                error_intervals.append(time.time() - last_error_time)
            last_error_time = time.time()
            current_delay *= 1.5 # Aumentar delay ante errores de conexión
            time.sleep(current_delay)

        # Si ya hemos encontrado suficientes 429 s o alcanzamos el límite de peticiones, detenemos
        if error_429_requests >= 10: # Puedes ajustar este número
            print("Se han recibido suficientes errores 429, deteniendo la prueba de umbral.")
            break

    total_requests = num_requests
    success_rate = (successful_requests / total_requests) * 100 if total_requests > 0 else 0
    error_429_rate = (error_429_requests / total_requests) * 100 if total_requests > 0 else 0

    print(f"\n--- Resumen para {endpoint} ---")
    print(f"Total de peticiones: {total_requests}")
    print(f"Peticiones exitosas: {successful_requests}")
    print(f"Peticiones con error 429: {error_429_requests}")
    print(f"Otras peticiones con error: {other_error_requests}")
    print(f"Tasa de éxito: {success_rate:.2f}%")
    print(f"Tasa de error 429: {error_429_rate:.2f}%")
    if error_intervals:
        print(f"Tiempo promedio entre errores (4xx): {sum(error_intervals)/len(error_intervals):.2f}s")
        print(f"Intervalos de error registrados: {', '.join([f'{i:.2f}s' for i in error_intervals])}")
    else:
        print("No se registraron intervalos de error entre peticiones.")

    results_objective_a.append({
        'endpoint': endpoint,
        'total_requests': total_requests,
        'successful_requests': successful_requests,
        'error_429_requests': error_429_requests,
        'other_error_requests': other_error_requests,
        'success_rate': success_rate,
        'error_429_rate': error_429_rate,
        'avg_error_interval': sum(error_intervals)/len(error_intervals) if error_intervals else 0
    })

In [ ]:
# Ejecutar pruebas para el Objetivo A en un endpoint específico
# Se recomienda elegir un endpoint que no tenga efectos colaterales.
# 'https://api.spotify.com/v1/me/player/currently-playing' es bueno para esto.
# También puedes probar con 'https://api.spotify.com/v1/search?q=test&type=track'
test_rate_limit("https://api.spotify.com/v1/albums/4aawyAB9vmqN3uQ7FjRGTy", HEADERS)
# test_rate_limit("https://api.spotify.com/v1/search?q=test&type=track", HEADERS)

# Objetivo B: Tiempo de Respuesta vs. Velocidad de Conexión y Errores

Este objetivo analiza cómo la velocidad de tu conexión a internet afecta el tiempo de respuesta y la cantidad de errores. Para simular diferentes velocidades de conexión, necesitarías una herramienta externa (como tc en Linux, o programas que limitan el ancho de banda). Aquí, nos centraremos en medir el impacto de la velocidad de conexión percibida por la red local, lo cual es un proxy.

Métricas a Medir para el Objetivo B
- Tiempo de espera al hacer una petición (latencia de red + procesamiento de API).
- Tasa de éxito en peticiones.
- Tasa de error en peticiones (4xx o 5xx).

In [ ]:
def test_connection_speed_impact(endpoints, headers,network_speed, num_iterations=10):
    print("\n--- Probando impacto de la velocidad de conexión ---")

    for endpoint in endpoints:
        print(f"\nRealizando pruebas para: {endpoint}")
        total_response_time = 0
        successful_requests = 0
        error_requests = 0

        for i in range(num_iterations):
            start_time = time.time()
            try:
                response = requests.get(endpoint, headers=headers)
                request_duration = time.time() - start_time
                total_response_time += request_duration

                status_code = response.status_code
                if 200 <= status_code < 300:
                    successful_requests += 1
                else:
                    error_requests += 1
                    print(f"  [{i+1}/{num_iterations}] Error {status_code} para {endpoint} en {request_duration:.4f}s")

            except requests.exceptions.RequestException as e:
                error_requests += 1
                print(f"  [{i+1}/{num_iterations}] Excepción de red: {e} para {endpoint}")

            time.sleep(0.5) # Pequeña pausa entre peticiones

        total_requests = num_iterations
        avg_response_time = total_response_time / successful_requests if successful_requests > 0 else 0
        success_rate = (successful_requests / total_requests) * 100 if total_requests > 0 else 0
        error_rate = (error_requests / total_requests) * 100 if total_requests > 0 else 0

        print(f"--- Resumen para {endpoint} (simulando una velocidad de conexión) ---")
        print(f"  Total de peticiones: {total_requests}")
        print(f"  Peticiones exitosas: {successful_requests}")
        print(f"  Peticiones con error (4xx/5xx): {error_requests}")
        print(f"  Tiempo de respuesta promedio (exitosas): {avg_response_time:.4f}s")
        print(f"  Tasa de éxito: {success_rate:.2f}%")
        print(f"  Tasa de error: {error_rate:.2f}%")

        results_objective_b.append({
            'endpoint': endpoint,
            'total_requests': total_requests,
            'successful_requests': successful_requests,
            'error_requests': error_requests,
            'avg_response_time': avg_response_time,
            'success_rate': success_rate,
            'error_rate': error_rate,
            'connection_speed_simulated': network_speed
        })

In [ ]:
# Ejecutar pruebas para el Objetivo B
# Nota: La "velocidad de conexión" en esta prueba es tu velocidad de red actual.
# Para simular diferentes velocidades, necesitarías una herramienta externa o correr el script en diferentes entornos.
test_connection_speed_impact(ENDPOINTS_A_PROBAR, HEADERS, '200 Mbps', num_iterations=20)

# Objetivo C: Tiempo de Respuesta vs. Región (VPN)

Para este objetivo, necesitarás una VPN (Virtual Private Network) que te permita cambiar tu ubicación geográfica. Deberás ejecutar el script desde diferentes ubicaciones VPN para observar la latencia y los errores.

Métricas a Medir para el Objetivo C
- Tiempo de espera al hacer una petición (latencia de red + procesamiento de API).
- Tasa de éxito en peticiones.
- Tasa de error en peticiones (4xx o 5xx).

In [ ]:
def test_region_impact(endpoints, headers, region_name, num_iterations=10):
    print(f"\n--- Probando impacto de la región: {region_name} ---")

    for endpoint in endpoints:
        print(f"\nRealizando pruebas para: {endpoint}")
        total_response_time = 0
        successful_requests = 0
        error_requests = 0

        for i in range(num_iterations):
            start_time = time.time()
            try:
                response = requests.get(endpoint, headers=headers)
                request_duration = time.time() - start_time
                total_response_time += request_duration

                status_code = response.status_code
                if 200 <= status_code < 300:
                    successful_requests += 1
                else:
                    error_requests += 1
                    print(f"  [{i+1}/{num_iterations}] Error {status_code} para {endpoint} en {request_duration:.4f}s (Región: {region_name})")

            except requests.exceptions.RequestException as e:
                error_requests += 1
                print(f"  [{i+1}/{num_iterations}] Excepción de red: {e} para {endpoint} (Región: {region_name})")

            time.sleep(0.5) # Pequeña pausa entre peticiones

        total_requests = num_iterations
        avg_response_time = total_response_time / successful_requests if successful_requests > 0 else 0
        success_rate = (successful_requests / total_requests) * 100 if total_requests > 0 else 0
        error_rate = (error_requests / total_requests) * 100 if total_requests > 0 else 0

        print(f"--- Resumen para {endpoint} en la región: {region_name} ---")
        print(f"  Total de peticiones: {total_requests}")
        print(f"  Peticiones exitosas: {successful_requests}")
        print(f"  Peticiones con error (4xx/5xx): {error_requests}")
        print(f"  Tiempo de respuesta promedio (exitosas): {avg_response_time:.4f}s")
        print(f"  Tasa de éxito: {success_rate:.2f}%")
        print(f"  Tasa de error: {error_rate:.2f}%")

        results_objective_c.append({
            'endpoint': endpoint,
            'region': region_name,
            'total_requests': total_requests,
            'successful_requests': successful_requests,
            'error_requests': error_requests,
            'avg_response_time': avg_response_time,
            'success_rate': success_rate,
            'error_rate': error_rate,
        })

In [ ]:
# --- Ejecución para el Objetivo C ---
# Para ejecutar esto, DEBES cambiar la región de tu VPN manualmente
# y luego ejecutar esta función para cada región.

print("\n--- ¡Preparado para probar impacto por región! ---")
print("Por favor, conéctate a tu VPN a la region de Argentina y luego ejecuta la siguiente línea:")
test_region_impact(ENDPOINTS_A_PROBAR, HEADERS, "Estados Unidos", num_iterations=20)

# Análisis y Visualización de Resultados

Una vez ejecutadas las pruebas, usamos pandas para organizar y analizar los datos de forma más sencilla.

In [ ]:
def analyze_results():
    df_a_results: pd.DataFrame | None = None
    df_b_results: pd.DataFrame | None = None
    df_c_results: pd.DataFrame | None = None

    if results_objective_a:
        df_a_results = pd.DataFrame(results_objective_a)
        df_a_results.to_csv("files/spotify_api_rate_limit_results.csv", index=False)

    if results_objective_b:
        df_b_results = pd.DataFrame(results_objective_b)
        df_b_results.to_csv("files/spotify_api_connection_speed_results.csv", index=False)

    if results_objective_c:
        df_c_results = pd.DataFrame(results_objective_c)
        df_c_results.to_csv("files/spotify_api_region_impact_results.csv", index=False)

    return df_a_results, df_b_results, df_c_results

In [ ]:
# analyze results
df_a, df_b, df_c = analyze_results()

In [ ]:
df_a